In [36]:
input_path = "/disk/dragon-storage/homes/seber007/data/rat/peptides/crux-output/crux-7-30-rat-peptides.txt"

all_peptides = []

with open(input_path) as crux_file:
    for line in crux_file:
        split_line = line.split("\t")

        peptide_sequence = split_line[0]

        all_peptides.append(peptide_sequence)     


# now shuffle the peptides and divide into 90% train/validation, and 10% test, and save to files

output_training_path = "/disk/dragon-storage/homes/seber007/data/rat/peptides/train_peptides_apr4.txt"
output_test_path = "/disk/dragon-storage/homes/seber007/data/rat/peptides/test_peptides_apr4.txt"


import random
from sklearn.model_selection import train_test_split
seed = 32

random.Random(32).shuffle(all_peptides)



train_set, test_set = train_test_split(all_peptides, train_size=0.9, random_state=32)


with open(output_training_path, "w+") as train_file:
    for peptide in train_set:
        train_file.write(f"{peptide}\n")


with open(output_test_path, "w+") as test_file:
    for peptide in test_set:
        test_file.write(f"{peptide}\n")


In [ ]:
# now load the file (for now only the train file) and put it in prosit-ready format
# for these initial tests (apr4) we will use only the precursor charges of +1, +2, +3, and a collision energy of 30

In [3]:

output_training_path = "/disk/dragon-storage/homes/seber007/data/rat/peptides/train_peptides_apr4.txt"
output_test_path = "/disk/dragon-storage/homes/seber007/data/rat/peptides/test_peptides_apr4.txt"

In [19]:
# create the prosit-formatted list of peptides and save to csv
prosit_lines = []

collision_energy = 30
precursor_charges = [1, 2, 3]

with open(output_training_path) as train_file:
    for peptide in train_file:
        peptide = peptide.strip("\n")

        for charge in precursor_charges:
            prosit_lines.append(f"{peptide},{collision_energy},{charge}")


import math

peptides_per_file = 100_000 # prosit can fail to download for large files so we split into batches of 100k
total_batches = math.ceil(len(prosit_lines) / peptides_per_file)
current_batch = 0

while current_batch < total_batches:
    out_file_name = f"/disk/dragon-storage/homes/seber007/data/rat/peptides/prosit_input_train_apr4_batch{current_batch}.txt"

    starting_index = current_batch * peptides_per_file

    with open(out_file_name, "w+") as prosit_outfile:
        for i in range(starting_index, min(starting_index + peptides_per_file, len(prosit_lines))):
            if i == starting_index:
                prosit_outfile.write("modified_sequence,collision_energy,precursor_charge\n")

            prosit_outfile.write(f"{prosit_lines[i]}\n")

    current_batch += 1
